In [2]:
import numpy as np
import pandas as pd

from keras import models
from keras import layers
from keras import optimizers
from keras import Input, layers  # for 7.1.1
from keras.models import Sequential, Model  # for 7.1.1

import rpy2.robjects as robjects # used to import .Rdata into python 

import cmath
import matplotlib.pyplot as plt

Reading images into Python environment

In [1]:
robjects.r['load']('features.RData')

features = robjects.r['features']
features = np.array(features)

df = pd.DataFrame(data=features[0:,0:],
           index=[i for i in range(features.shape[0])],
           columns=['x'+str(i) for i in range(features.shape[1])])


from skimage.io import imread_collection
Data_dir = "Data/*/*.jpg"
imdata = imread_collection(Data_dir)

imdata[0] #  to access images 0,1,....25330
type(imdata)
type(imdata[0])
imdata[0].shape


KeyboardInterrupt



Store images as tensors

In [3]:
im_array = np.empty([25331,36,48,3])
im_array[0,:,:,:].shape

from skimage.transform import resize

for i in range(25331):
    img_resized = resize(imdata[i], output_shape= (36,48,3))
    im_array[i,:,:,:] = img_resized
    if i%1000 == 0:
        print(i)

In [ ]:
np.save("im_array.npy", im_array)

Split into training and validation sets

In [ ]:
labels = ['AK'] * 867 + ['BCC'] * 3323 + ['BKL']*2624 + ['DF']*239 + ['MEL']*4522 + ['NV']*12875 + ['SCC']*628 + ['VASC']*253
len(labels)

from sklearn.model_selection import train_test_split

train, test, label_train, label_test  = train_test_split(im_array, labels, test_size=0.25, random_state=42)

np.save("label_train.npy", label_train)
np.save("label_test.npy", label_test)

Encoder Architecture

In [4]:
input_img = Input(shape=(36, 48, 3))


x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img) # 36x48x16
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(4, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((3, 3), padding='valid')(x)

In [ ]:
encoded

Decoder Architecture

In [ ]:
x = layers.Conv2D(4, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((3, 3))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)

decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

In [ ]:
decoded

Define model, loss function, optimzation algorithm

In [ ]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

Fit the model to the training data

In [ ]:
autoencoder.fit(train, train, epochs=10, batch_size=512, shuffle=True, validation_data=(test,test))

Predict using the fitted model

In [ ]:
decoded_imgs = autoencoder.predict(test)

In [ ]:
plt.imshow(test[2]) # original image

In [ ]:
plt.imshow(decoded_imgs[2]) # reconstructed image

Encode the training images

In [ ]:
encoder = Model(input_img, encoded)
encoded_train = encoder(train)

encoded_train.shape # encoded features

In [ ]:
encoded_train_array = np.asarray(encoded_train)
encoded_train_array.shape
np.save("encoded_train_new.npy", encoded_train_array )

Encode validation set as well

In [ ]:
encoded_test = encoder(test)
encoded_test_array = np.asarray(encoded_test)
encoded_test_array.shape

np.save("encoded_test_new.npy", encoded_test_array )